In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import time

import CustomHyperModel
import EnergyPricesLibrary as Ep

from kerastuner.tuners import BayesianOptimization

%load_ext autoreload
%autoreload 2

In [2]:
def make_predictions(model,scaler_y,trainX,trainY,testX,testY,n_steps_out,len_output_features):
    
    # make predictions
    trainPredict = model.predict(trainX)
    trainPredict = trainPredict.reshape(trainPredict.shape[0]*n_steps_out,len_output_features)
    testPredict  = model.predict(testX)
    testPredict  = testPredict.reshape(testPredict.shape[0]*n_steps_out,len_output_features)
    
    # invert predictions
    trainPredict = scaler_y.inverse_transform(trainPredict)
    trainY_ = scaler_y.inverse_transform(trainY.reshape(trainY.shape[0]*n_steps_out,len_output_features))
    
    testPredict = scaler_y.inverse_transform(testPredict)
    testY_ = scaler_y.inverse_transform(testY.reshape(testY.shape[0]*n_steps_out,len_output_features))
        
    return trainPredict,trainY_,testPredict,testY_

def get_metrics(trainY,trainPredict,testY,testPredict):
    
    trainMAPE  = Ep.MAPE(trainPredict,trainY)
    testMAPE  = Ep.MAPE(testPredict,testY)
    
    train_sMAPE  = Ep.sMAPE(trainY,trainPredict)
    test_sMAPE  = Ep.sMAPE(testY,testPredict)
    
    return trainMAPE,testMAPE,train_sMAPE,test_sMAPE

# DATASET

## Load Dataset

In [3]:
data_horaria_path = os.path.join('..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Horaria.xlsx')
data_horaria = pd.read_excel(data_horaria_path)
data_horaria = data_horaria.set_index('Fecha')

In [4]:
precio_bolsa_path = os.path.join('..','..','dataset','Series','Sabanas','Original','Sabana_Datos_Precio_Bolsa.xlsx')
precio_bolsa = pd.read_excel(precio_bolsa_path)
precio_bolsa = precio_bolsa.set_index('Fecha')

In [5]:
nombre_series_horaria = data_horaria.columns

In [6]:
data_horaria_full = pd.concat([data_horaria,precio_bolsa],axis=1)
data_horaria_full.shape,data_horaria.shape,precio_bolsa.shape

((177480, 86), (177480, 85), (177480, 1))

## Build Window

In [7]:
d = 'All'
start_date_train = '2000-02-01'
start_date_val = '2020-01-01'
start_date_test = '2020-04-01'
end_date_test = '2020-05-01'
n_steps_out=24
output_columns = ['$kWh']

In [8]:
n_steps_in = 720
overlap = 24
inputs_columns = nombre_series_horaria

len_input_features = len(inputs_columns)
len_output_features = len(output_columns)

results = Ep.SplitTimeseriesMultipleTimesBackAhead(df=data_horaria_full,
                                                   day=d,
                                                   start_date_train=start_date_train,start_date_val=start_date_val,
                                                   start_date_test=start_date_test,end_date_test=end_date_test,
                                                   n_steps_out=n_steps_out,n_steps_in=n_steps_in,overlap=overlap,
                                                   input_features=inputs_columns,output_features=output_columns)

trainX_H,trainY_H,valX_H,valY_H,testX_H,testY_H,scaler_H_x,scaler_H_y,df2_H,dataset_H = results

In [9]:
trainX_H.shape,trainY_H.shape,valX_H.shape,valY_H.shape,testX_H.shape,testY_H.shape

((7244, 720, 85),
 (7244, 24, 1),
 (91, 720, 85),
 (91, 24, 1),
 (30, 720, 85),
 (30, 24, 1))

In [10]:
callback_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                          factor=0.1,
                                                          min_lr=1e-5,
                                                          patience=5,
                                                          verbose=1)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=10,
                                                  mode='min')

callbacks = [callback_reduce_lr,early_stopping]

In [11]:
INPUT_SHAPE = (n_steps_in,len_input_features)

arquitectura31 = CustomHyperModel.Arquitectura31(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura32 = CustomHyperModel.Arquitectura32(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura33 = CustomHyperModel.Arquitectura33(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura34 = CustomHyperModel.Arquitectura34(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura35 = CustomHyperModel.Arquitectura35(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura36 = CustomHyperModel.Arquitectura36(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura37 = CustomHyperModel.Arquitectura37(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura38 = CustomHyperModel.Arquitectura38(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)

arquitectura39 = CustomHyperModel.Arquitectura39(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura40 = CustomHyperModel.Arquitectura40(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura41 = CustomHyperModel.Arquitectura41(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura42 = CustomHyperModel.Arquitectura42(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura43 = CustomHyperModel.Arquitectura43(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura44 = CustomHyperModel.Arquitectura44(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura45 = CustomHyperModel.Arquitectura45(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)
arquitectura46 = CustomHyperModel.Arquitectura46(input_shape=INPUT_SHAPE,n_steps_out=n_steps_out)

In [12]:
arq_list = [arquitectura31,arquitectura32,arquitectura33,arquitectura34,
            arquitectura35,arquitectura36,arquitectura37,arquitectura38,
            arquitectura39,arquitectura40,arquitectura41,arquitectura42,
            arquitectura43,arquitectura44,arquitectura45,arquitectura46]

In [ ]:
arq_idx = 31
arq_best_models = dict()

for arq in arq_list:
    
    bayesian_tuner = BayesianOptimization(
        arq,
        objective='val_loss',
        num_initial_points=1,
        max_trials=10,
        directory='my_dir',
        project_name=str(arq_idx)
    )
    
    # Overview of the task
    bayesian_tuner.search_space_summary()
    
    # Performs the hyperparameter tuning
    search_start = time.time()
    bayesian_tuner.search(x=trainX_H,y=trainY_H,
                      epochs=200,
                      validation_data=(valX_H,valY_H),
                      callbacks=callbacks)
    search_end = time.time()
    elapsed_time = search_end - search_start
    
    print('Tiempo Total Transcurrido {}'.format(elapsed_time))
        
    dict_key = 'Arquitectura {}'.format(arq_idx)

    arq_best_models[dict_key] = dict()
    bs_model = bayesian_tuner.oracle.get_best_trials(1)[0]
    
    model = bayesian_tuner.get_best_models(num_models=1)[0]
    
    trainPredict,trainY_true,testPredict,testY_true = make_predictions(model,scaler_H_y,trainX_H,trainY_H,valX_H,valY_H,
                                                                       n_steps_out,len_output_features)
    
    trainMAPE,testMAPE,train_sMAPE,test_sMAPE = get_metrics(trainY_true,trainPredict,testY_true,testPredict)

    arq_best_models[dict_key]['Score'] = bs_model.score
    arq_best_models[dict_key]['Tiempo Scaneo'] = elapsed_time
    arq_best_models[dict_key]['Mape Train'] = trainMAPE
    arq_best_models[dict_key]['Mape Test'] = testMAPE
    arq_best_models[dict_key]['sMape Train'] = train_sMAPE
    arq_best_models[dict_key]['sMape Test'] = test_sMAPE

    if bs_model.hyperparameters.values:
        for hp, value in bs_model.hyperparameters.values.items():
            arq_best_models[dict_key][hp] = value
    
    arq_idx += 1

INFO:tensorflow:Reloading Oracle from existing project my_dir\31\oracle.json
Search space summary
Default search space size: 10
lstm_units_layer_1 (Int)
{'default': 128, 'conditions': [], 'min_value': 64, 'max_value': 512, 'step': 64, 'sampling': None}
kernel_regularizer_layer_1 (Float)
{'default': 0.01, 'conditions': [], 'min_value': 0.0, 'max_value': 0.105, 'step': 0.0075, 'sampling': None}
dropout_regularizer_layer_1 (Float)
{'default': 0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.99, 'step': 0.09, 'sampling': None}
lstm_units_layer_2 (Int)
{'default': 128, 'conditions': [], 'min_value': 64, 'max_value': 512, 'step': 64, 'sampling': None}
kernel_regularizer_layer_2 (Float)
{'default': 0.01, 'conditions': [], 'min_value': 0.0, 'max_value': 0.105, 'step': 0.0075, 'sampling': None}
dropout_regularizer_layer_2 (Float)
{'default': 0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.99, 'step': 0.09, 'sampling': None}
dense_layer (Int)
{'default': 48, 'conditions': [], 'min_val

In [ ]:
import json
with open('BestModels-H720-V7.json', 'w') as outfile:
    json.dump(arq_best_models, outfile)

In [ ]:
arq_best_models